In [43]:
import os 
from pathlib import Path
abspath = Path('C:\\Users\\harkum\\Documents\\6311\\Software')
os.chdir(abspath)

<h2 style=' color: 	#0bff00;border-bottom: 2px dotted #ff0000;'> Register address parsing</h2>

<h3 style='color:#ffac00'> Parse all the register addres 0x pattrens</h3>

In [1]:
from typing import List
import json
import random
import re 
import pandas as pd 
register_instructions = {}
i=0
for sh in ['Digital','Reference','Boost','DAC+PA','AZ_COMP']:
    sheet = 'raw_data/'+sh+'_data.csv'
    data = pd.read_csv(sheet)
    for col in data.columns.to_list()[1:]:
        instructions = data[col][1].split('\n')
        for instruction in instructions:
            if re.match('0x', instruction.lower()):
                register_instructions.update({i : {
                    'page':sh,
                    'dft':col,
                    'ins':instruction
                }})
            i+=1
print(register_instructions)
with open('raw_data/registers_parsing_raw_dump.json', 'w') as file:
    json.dump(register_instructions, file)

{1: {'page': 'Digital', 'dft': 'BIST', 'ins': '0xFE_0x00 "Page 0"'}, 2: {'page': 'Digital', 'dft': 'BIST', 'ins': '0x11_0x00 "Set TDM mode"'}, 3: {'page': 'Digital', 'dft': 'BIST', 'ins': '0xFE_0x01 "Page 1"'}, 4: {'page': 'Digital', 'dft': 'BIST', 'ins': '0x06_0x03 "Bist enable=1"'}, 10: {'page': 'Digital', 'dft': 'BIST', 'ins': '0x06_0x00 "Bist enable=0"'}, 14: {'page': 'Digital', 'dft': 'PAD LOOPBACK', 'ins': '0x00_0x0 "Set IO test mode"'}, 15: {'page': 'Digital', 'dft': 'PAD LOOPBACK', 'ins': '0x00_0x0 "Set IO enable=1"'}, 17: {'page': 'Digital', 'dft': 'PAD LOOPBACK', 'ins': '0x00_0x00 "Set IO enable=0"'}, 21: {'page': 'Digital', 'dft': 'SCAN', 'ins': '0x05_0x01 "Set scan mode"'}, 26: {'page': 'Reference', 'dft': 'UVLO', 'ins': '0xFE_0x01 "Select page 1"'}, 27: {'page': 'Reference', 'dft': 'UVLO', 'ins': '0x2F_0xAA "Unlock test page"'}, 31: {'page': 'Reference', 'dft': 'UVLO', 'ins': '0x1A_0x00 "UVLO"'}, 35: {'page': 'Reference', 'dft': 'UVLO', 'ins': '0x2F_0xBB "Lock test page"'}

<h3 style = 'color:#00f6ff'> Extract the register address</h3>

In [2]:
pattern1=re.compile(r"\b(0[xX]+[0-9a-fA-F]+)+\[(.*?)\]+[_]+(0[xX]+[0-9a-fA-F]+)\b")
if re.findall(pattern1, '0x00[1:2]_0x0')[0]:
    print(re.findall(pattern1, '0x00[1:2]_0x0')[0])

('0x00', '1:2', '0x0')


In [3]:
def extract_RegisterAddress__Instruction(instrunction: str):
    pattern1 = re.compile(r"\b(0[xX]+[0-9a-fA-F]+)+\[(.*?)\]+[_]+(0[xX]+[0-9a-fA-F]+)\b") # sample data 0x00[1:2]_0x0
    pattern2 = re.compile(r"\b(0[xX]+[0-9a-fA-F]+)+[_]+(0[xX]+[0-9a-fA-F]+)\b") # sample data 0x00_0x0
    pattern3 = re.compile(r"\b(0[xX]+[0-9a-fA-F]+)+\[(.*?)\]") # sample data 0x00[1:2]
    # parse the data for the bit information and the register information 
    
    def register_format(instrunction):
        instruction_length = len(instrunction)
        register = {}
        # there is no bit field it is the default size
        if instruction_length == 2:
            if ":" in instrunction[1]:
                bits = instrunction[1].split(':')
                register={
                "RegAddr" : hex(int(instrunction[0],16)),
                "MSB" :hex(int(bits[1],16)),
                "LSB" : hex(int(bits[0],16)),
                "Data" : None,
                }
            else:
                register={
                    "RegAddr" : hex(int(instrunction[0],16)),
                    "MSB" : 7,
                    "LSB" : 0,
                    "Data" : hex(int(instrunction[1],16)),
                }
        # check for the bit field 
        elif instruction_length == 3:
            #check for the single bit or bit field 
            if ":" in instrunction[1]:
                bits = instrunction[1].split(':')
                register={
                "RegAddr" : hex(int(instrunction[0],16)),
                "MSB" :hex(int(bits[1],16)),
                "LSB" : hex(int(bits[0],16)),
                "Data" : hex(int(instrunction[2],16)),
                }

            else:
                register={
                "RegAddr" : hex(int(instrunction[0],16)),
                "MSB" :hex(int(instrunction[1],16)),
                "LSB" : hex(int(instrunction[1],16)),
                "Data" : hex(int(instrunction[2],16)),
                }
        return register
    # Remove the comments from the register
    if re.search(r"\"(.*?)\"",instrunction):
        instrunction = re.sub(r"\"(.*?)\"",'',instrunction)
    
    
    if  re.match(pattern1, instrunction):
        # register = register_format(value)
        register = register_format(re.findall(pattern1, instrunction)[0])
        # parse only register information and the register value 
    elif re.match(pattern2, instrunction): 
        register = register_format(re.findall(pattern2, instrunction)[0])

       
    elif re.match(pattern3, instrunction): 
        register = register_format(re.findall(pattern3, instrunction)[0])
    else: 
        register = {}

    return register
extract_RegisterAddress__Instruction('0x00[1:2]')

{'RegAddr': '0x0', 'MSB': '0x2', 'LSB': '0x1', 'Data': None}

<h3 style='color: #fff100'> Test the Register address extraction </h3>

In [4]:
clean_registers = {}
for i,instruction_set in register_instructions.items():
    # print(instruction_set)
    instruction = instruction_set.get('ins')
    if register_instruction_parse := extract_RegisterAddress__Instruction(instruction):
        # print(register_instruction_parse)
        clean_registers.update({
            i:{
                'set':instruction_set,
                'clean':register_instruction_parse
            }
        })
    else:
        print(instruction)
        clean_registers.update({
            i:None
        })
with open('raw_data/clean_registers.json', 'w') as file:
    json.dump(clean_registers, file)

<h2 style='color:  	#ffaa00; border-bottom: 2px dotted #ff00aa'> 2. Parse TrimSweep </h2>

Parse Trimsweep register address <b>syntax :</b> ``` TrimSweep - register address [MSB:LSB] ``` ; <i>example :~</i> <b style="color:#aaff00">TrimSweep - 0xEF[4:0]</b> 

<h3 style='color:#ff00aa'> Collect all the TrimSweep Instructions</h3>

In [8]:
from typing import List
import json
import random
import re 
import pandas as pd 
trimsweep_instructions = {}
i=0
for sh in ['Digital','Reference','Boost','DAC+PA','AZ_COMP']:
    sheet = 'raw_data/'+sh+'_data.csv'
    data = pd.read_csv(sheet)
    for col in data.columns.to_list()[1:]:
        instructions = data[col][1].split('\n')
        for instruction in instructions:
            if re.match('trimsweep', instruction.lower()):
                trimsweep_instructions.update({i : {
                    'page':sh,
                    'dft':col,
                    'ins':instruction
                }})
                i+=1
print(trimsweep_instructions)
with open('raw_data/trimsweep_instructions_parsing_raw_dump.json', 'w') as file:
    json.dump(trimsweep_instructions, file)

{0: {'page': 'Reference', 'dft': 'VBGR_ADJ_TRIM', 'ins': 'TrimSweep - 0xB0[7:4] "Select code which sets ATEST voltage as close as possible to target"'}, 1: {'page': 'Reference', 'dft': 'TSDN', 'ins': 'TrimSweep - 0xB3[7:5] "Select code which sets ATEST voltage as close as possible to target"'}, 2: {'page': 'Reference', 'dft': 'FRO_CLOCK', 'ins': 'TrimSweep - 0xEF[4:0] "Select code which sets ATEST frequency as close as possible to target"'}, 3: {'page': 'Boost', 'dft': 'BST_OCP_TRIM', 'ins': 'TrimSweep - 0xB2[5]_0xB1[3:0] "trim in 2 different registers, what is the correct command"'}}


<h3 style='color:#00aaff'> Extract the TrimSweep Register address</h3>

In [36]:
pattern = re.compile(r"\b(0[xX]+[0-9a-fA-F]+)+\[(.*?)\]+(0[xX]+[0-9a-fA-F]+)+\[(.*?)\]")
re.findall(pattern, '0xB2[5]0xB1[3:0]')[0]

('0xB2', '5', '0xB1', '3:0')

In [14]:
def extract_TrimSweep__Instruction(instruction: str):
    def register_format(instrunction):
        instruction_length = len(instrunction)
        register = {}
        # there is no bit field it is the default size
        if instruction_length == 2:
            if ":" in instrunction[1]:
                bits = instrunction[1].split(':')
                register={
                "RegAddr" : hex(int(instrunction[0],16)),
                "MSB" :hex(int(bits[1],16)),
                "LSB" : hex(int(bits[0],16)),
                "Data" : None,
                }
            else:
                register={
                    "RegAddr" : hex(int(instrunction[0],16)),
                    "MSB" : 7,
                    "LSB" : 0,
                    "Data" : hex(int(instrunction[1],16)),
                }
        # check for the bit field 
        elif instruction_length == 3:
            #check for the single bit or bit field 
            if ":" in instrunction[1]:
                bits = instrunction[1].split(':')
                register={
                "RegAddr" : hex(int(instrunction[0],16)),
                "MSB" :hex(int(bits[1],16)),
                "LSB" : hex(int(bits[0],16)),
                "Data" : hex(int(instrunction[2],16)),
                }

            else:
                register={
                "RegAddr" : hex(int(instrunction[0],16)),
                "MSB" :hex(int(instrunction[1],16)),
                "LSB" : hex(int(instrunction[1],16)),
                "Data" : hex(int(instrunction[2],16)),
                }
        return register
    
    pattern = re.compile(r"\b(0[xX]+[0-9a-fA-F]+)+\[(.*?)\]")
    instruction = re.findall(pattern, instruction)[0]
    return register_format(instrunction=instruction)
extract_TrimSweep__Instruction('TrimSweep - 0xB0[7:4] "Select code which sets ATEST voltage as close as possible to target"')

{'RegAddr': '0xb0', 'MSB': '0x4', 'LSB': '0x7', 'Data': None}

<h3 style='color: 	#aa00ff'> Test TrimSweep Register Address Extraction </h3>

 <b style='color:#aaff00'> Failed :</b> <b style='color: red'>    "set": {
      "page": "Boost",
      "dft": "BST_OCP_TRIM",
      "ins": "TrimSweep - 0xB2[5]_0xB1[3:0] \"trim in 2 different registers, what is the correct command\""
    }, </b>

In [17]:
clean_trimsweep = {}
for i,instruction_set in trimsweep_instructions.items():
    # print(instruction_set)
    instruction = instruction_set.get('ins')
    if register_instruction_parse := extract_TrimSweep__Instruction(instruction):
        # print(register_instruction_parse)
        clean_trimsweep.update({
            i:{
                'set':instruction_set,
                'clean':register_instruction_parse
            }
        })
    else:
        print(instruction)
        clean_trimsweep.update({
            i:None
        })
with open('raw_data/clean_trimsweep.json', 'w') as file:
    json.dump(clean_trimsweep, file)

<h2 style='color:  	#ffaa00; border-bottom: 2px dotted #ff00aa'> 3. Parse Trim Register Address </h2>

Parse Trimsweep register address <b>syntax :</b> ``` Trim -register address[MSB:LSB] ``` ; <i>example :~</i> <b style="color:#aaff00">Trim - 0xB3[7:5] </b> 

<h3 style='color:#ff00aa'> Collect all the Trim Instructions</h3>

In [37]:
from typing import List
import json
import random
import re 
import pandas as pd 
trim_instructions = {}
i=0
for sh in ['Digital','Reference','Boost','DAC+PA','AZ_COMP']:
    sheet = 'raw_data/'+sh+'_data.csv'
    data = pd.read_csv(sheet)
    for col in data.columns.to_list()[1:]:
        instructions = data[col][1].split('\n')
        for instruction in instructions:
            if re.match('trim -', instruction.lower()):
                trim_instructions.update({i : {
                    'page':sh,
                    'dft':col,
                    'ins':instruction
                }})
                i+=1
print(trim_instructions)
with open('raw_data/trim_instructions_parsing_raw_dump.json', 'w') as file:
    json.dump(trim_instructions, file)

{0: {'page': 'Reference', 'dft': 'VBGR_ADJ_TRIM', 'ins': 'Trim - 0xB0[7:4] '}, 1: {'page': 'Reference', 'dft': 'TSDN', 'ins': 'Trim - 0xB3[7:5] '}, 2: {'page': 'Reference', 'dft': 'FRO_CLOCK', 'ins': 'Trim - 0xEF[4:0] '}, 3: {'page': 'Boost', 'dft': 'BST_OCP_TRIM', 'ins': 'Trim - 0xB2[5]0xB1[3:0]'}}


In [41]:
def extract_Trim__Instruction(instruction: str):
    return extract_TrimSweep__Instruction(instruction)
extract_Trim__Instruction('Trim - 0xB0[7:4] ')

{'RegAddr': '0xb0', 'MSB': '0x4', 'LSB': '0x7', 'Data': None}

In [42]:
clean_trim = {}
for i,instruction_set in trim_instructions.items():
    # print(instruction_set)
    instruction = instruction_set.get('ins')
    if register_instruction_parse := extract_Trim__Instruction(instruction):
        # print(register_instruction_parse)
        clean_trim.update({
            i:{
                'set':instruction_set,
                'clean':register_instruction_parse
            }
        })
    else:
        print(instruction)
        clean_trim.update({
            i:None
        })
with open('raw_data/clean_trim.json', 'w') as file:
    json.dump(clean_trim, file)